Installing Dependencies

In [27]:
%%capture
!pip install git-python==1.0.3
!pip install sacrebleu==1.4.2
!pip install rouge_score
!pip install farasapy
!git clone https://github.com/aub-mind/arabert
!pip install pyarabic
!pip install datasets
!pip install -U transformers==4.5.1
!git clone https://github.com/aub-mind/Arabic-Empathetic-Chatbot

Getting the dataset from Github

In [ ]:
!wget https://raw.githubusercontent.com/aub-mind/Arabic-Empathetic-Chatbot/master/arabic-empathetic-conversations.csv

Importing Necessary Libraries

In [8]:
import os
import numpy as np
import pandas as pd
from datasets import load_dataset 
from transformers import AutoTokenizer
from transformers import EncoderDecoderModel
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from sacrebleu import corpus_bleu

Specifying the encoder/decoder maximum sequence length and the BERT model name

In [9]:
encoder_max_length=150
decoder_max_length=150
model_name = "aubmindlab/bert-base-arabert"

Specificying the AraBERT tokenizer

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading the dataset using ArabicEmpatheticDialogues.py

In [ ]:
all_data = load_dataset("ArabicEmpatheticDialogues.py")
train_data = all_data['train'].train_test_split(test_size=0.1,seed=42)['train']
val_data = all_data['train'].train_test_split(test_size=0.1,seed=42)['test']
dev_data = val_data.train_test_split(test_size=0.5,seed=42)['train']
test_data = val_data.train_test_split(test_size=0.5,seed=42)['test']

In [12]:
#Lengths of train/dev/test sets
print("Length of train data",len(train_data))
print("Length of dev data",len(dev_data))
print("Length of test data",len(test_data))

Length of train data 32965
Length of dev data 1831
Length of test data 1832


Define function to process data in model input format

In [19]:
def process_data_to_model_inputs(batch):                                                               
    # Tokenizer will automatically set [BOS] <text> [EOS]                                               
    inputs = tokenizer(batch["context"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["response"], padding="max_length", truncation=True, max_length=decoder_max_length)
                                                                                                        
    batch["input_ids"] = inputs.input_ids                                                               
    batch["attention_mask"] = inputs.attention_mask                                                     
    batch["decoder_input_ids"] = outputs.input_ids                                                      
    batch["labels"] = outputs.input_ids.copy()                                                          
    # mask loss for padding                                                                             
    batch["labels"] = [                                                                                 
        [-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]
    ]                     
    batch["decoder_attention_mask"] = outputs.attention_mask                                                                              
                                                                                                         
    return batch  

Specify Batch Size

In [16]:
batch_size=16

Prepare the train/dev/test sets

In [ ]:
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["context", "response"],
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

dev_data = dev_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["context", "response"],
)
dev_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

test_data = test_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["context", "response"],
)
test_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

Create the BERT2BERT architecture initialized with AraBERT pre-trained checkpoints

In [ ]:
arabert2arabert = EncoderDecoderModel.from_encoder_decoder_pretrained(model_name, model_name, tie_encoder_decoder=False)

Specifying parameters

In [22]:
#set special tokens
arabert2arabert.config.decoder_start_token_id = tokenizer.cls_token_id                                             
arabert2arabert.config.eos_token_id = tokenizer.sep_token_id
arabert2arabert.config.pad_token_id = tokenizer.pad_token_id

#sensible parameters for beam search
#set decoding params                               
arabert2arabert.config.max_length = 64
arabert2arabert.config.early_stopping = True

arabert2arabert.config.num_beams = 1
arabert2arabert.config.vocab_size = arabert2arabert.config.encoder.vocab_size

Specifying Metrics

In [23]:
import torch
import torch.nn as nn

def compute_metrics(pred):
  labels_ids = pred.label_ids
  #pred_ids = torch.argmax(pred.predictions,dim=2)
  pred_ids = pred.predictions  

  # all unnecessary tokens are removed
  pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  labels_ids[labels_ids == -100] = tokenizer.pad_token_id
  label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

  return {"bleu": round(corpus_bleu(pred_str , [label_str]).score, 4)}

Specifying Training Arguments (Not tuned)

In [24]:
#Set training arguments 
training_args = Seq2SeqTrainingArguments(
    output_dir="./model",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size//2,
    gradient_accumulation_steps = 2,
    predict_with_generate=True,
    do_eval=True,
    evaluation_strategy ="epoch",
    do_train=True,
    logging_steps=500,  
    save_steps= 32965 // ( batch_size * 2),  
    warmup_steps=100,
    eval_steps=10,
    #max_steps=16, # delete for full training
    num_train_epochs=3,# uncomment for full training
    overwrite_output_dir=True,
    save_total_limit=0,
    #fp16=True, 
)

Instantiate the trainer

In [25]:
trainer = Seq2SeqTrainer(
    model=arabert2arabert,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=dev_data,
    tokenizer=tokenizer
)

Train the BERT2BERT model

In [ ]:
trainer.train()

Evaluate the BERT2BERT model

In [ ]:
trainer.evaluate()

Save & Reload the BERT2BERT model

In [ ]:
trainer._save("./arabert2arabert")
tokenizer.save_pretrained("./arabert2arabert")

('./arabert2arabert/tokenizer_config.json',
 './arabert2arabert/special_tokens_map.json',
 './arabert2arabert/vocab.txt',
 './arabert2arabert/added_tokens.json')

## Chat with the model

In [ ]:
#Import pipeline
from transformers import pipeline

chatbot = pipeline("text2text-generation",model='./arabert2arabert')

In [ ]:
chatbot("مرحبا كيفك ؟",
    num_beams=10,
    max_length=200,
    top_p=0.9,
    repetition_penalty = 3.0,
    no_repeat_ngram_size = 3)

[{'generated_text': 'ايه بس ما في شي ؟'}]